In [1]:
import gc
import os
import random
import time
import torch
import datetime
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics.regression import R2Score
from tqdm import tqdm
import torch.nn.functional as F
import hickle as hkl
from torch.nn import AvgPool1d 
import torch.nn as nn
import gc
from glob import glob
from torch.nn.parallel import DistributedDataParallel as DDP


DATA_PATH = 'autodl-tmp/'


file_list = glob(DATA_PATH+'CDFrag/'+'c_data_frag_x*hkl')

file_list_mix = glob(DATA_PATH+'CDFrag L/'+'c_data_frag_x_*_1e-15.hkl')

In [2]:
len(file_list)

34296

In [3]:
len(file_list_mix)

96672

In [4]:
file_list[0]

'autodl-tmp/CDFrag/c_data_frag_x_1_2_0_1e-20.hkl'

In [5]:
file_list_mix[0]

'autodl-tmp/CDFrag L/c_data_frag_x_7_1_0_1e-15.hkl'

In [6]:
len('autodl-tmp/CDFrag/c_data_frag_')

30

In [7]:
len('autodl-tmp/CDFrag L/c_data_frag_')

32

In [8]:
zeroout_index = list(range(145,147))

zeroout_index = torch.tensor(list(set(zeroout_index)))

In [9]:
import gc
import os
import random
import time
import torch
import datetime
import numpy as np
import pandas as pd
# import polars as pl
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics.regression import R2Score
from tqdm import tqdm
import torch.nn.functional as F
import hickle as hkl
from torch.nn import AvgPool1d 
import torch.nn as nn
import gc
import glob
import random
import math

from torch.nn import LSTM, Conv1d, GRU, TransformerEncoder, TransformerEncoderLayer, BatchNorm1d


from torch.nn import LayerNorm

from matplotlib.pyplot import plot
# tmp = addition_y[10:774144:384,10]
# tmp = addition_y[:,2]


BATCH_SIZE = 384*6*3
MIN_STD = 1e-10
SCHEDULER_PATIENCE = 6
SCHEDULER_FACTOR = 10**(-0.2)
EPOCHS = 70
PATIENCE = 6
PRINT_FREQ = 300
BIN_NUM = 10
n_rounds = 1

In [10]:



seq_fea_list = ['state_t','state_q0001','state_q0002','state_q0003','state_u','state_v','pbuf_ozone','pbuf_CH4','pbuf_N2O']
num_fea_list = ['state_ps','pbuf_SOLIN','pbuf_LHFLX','pbuf_SHFLX','pbuf_TAUX','pbuf_TAUY','pbuf_COSZRS','cam_in_ALDIF','cam_in_ALDIR','cam_in_ASDIF','cam_in_ASDIR','cam_in_LWUP','cam_in_ICEFRAC','cam_in_LANDFRAC','cam_in_OCNFRAC','cam_in_SNOWHLAND']


seq_y_list = ['ptend_t','ptend_q0001','ptend_q0002','ptend_q0003','ptend_u','ptend_v']
num_y_list = ['cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC','cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD']

seq_fea_expand_list = []
for i in seq_fea_list:
    for j in range(60):
        seq_fea_expand_list.append(i+'_'+str(j))


seq_y_expand_list = []
for i in seq_y_list:
    for j in range(60):
        seq_y_expand_list.append(i+'_'+str(j))
        
norm_dict = dict()
TARGET_COLS = seq_y_expand_list + num_y_list
FEAT_COLS = seq_fea_expand_list + num_fea_list


In [11]:
len(FEAT_COLS)

556

In [12]:

class FFNN_LSTM_6_AVG_PLE(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_LSTM_6_AVG_PLE, self).__init__()
        
        self.encode_dim = 280
        self.hidden_dim = 260
        self.iter_dim = 1024
        self.num_emb = 8
        
        self.PiecewiseLinearEmbeddings = PiecewiseLinearEmbeddings(bins, self.num_emb-1, activation=False)

        self.LSTM_1 = LSTM(self.encode_dim,self.hidden_dim,6,batch_first=True,dropout=0.05,bidirectional=True)
        self.input_size = input_size
        self.Linear_1 = nn.Linear(self.num_emb*len(seq_fea_list)+self.num_emb*len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(6*self.hidden_dim+self.encode_dim, self.iter_dim)
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        self.Linear_4_0 = nn.Linear(self.iter_dim+self.num_emb*len(num_fea_list), self.iter_dim*2)

        self.Linear_4 = nn.Linear(self.iter_dim*2, len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        
        x_ple = self.PiecewiseLinearEmbeddings(x)
        x = x[:,:,None]
        x = torch.concat((x,x_ple),dim=-1)
        
         
        # print(x_ple.shape)
        
        x_seq = x[:,0:60*len(seq_fea_list),:]
        
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60,self.num_emb))
        x_seq = torch.transpose(x_seq, 1, 2)
        x_seq = x_seq.reshape((-1,60,len(seq_fea_list)*self.num_emb))
        
        x_num = x[:,60*len(seq_fea_list):x.shape[1],:]
        x_num = x_num.reshape((-1,len(num_fea_list)*self.num_emb))

        x_num_repeat = x_num[:,None,:]
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/5))
        
        x_seq_1,_ = self.LSTM_1(x_seq/5)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))

        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/5))
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_num_out = torch.concat((torch.mean(x_seq_1,dim=1),x_num),dim=-1)
        
        x_num_out = F.elu(self.Linear_4_0(x_num_out))
        x_num_out = self.Linear_4(x_num_out)
        
        # print(self.weight)
        
        return self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3


seq_fea_expand_list = []
for i in seq_fea_list:
    for j in range(60):
        seq_fea_expand_list.append(i+'_'+str(j))


seq_y_expand_list = []
for i in seq_y_list:
    for j in range(60):
        seq_y_expand_list.append(i+'_'+str(j))
        
norm_dict = dict()
TARGET_COLS = seq_y_expand_list + num_y_list
FEAT_COLS = seq_fea_expand_list + num_fea_list

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class NumpyDataset(Dataset):
    def __init__(self, x, y):
        """
        Initialize with NumPy arrays.
        """
        assert x.shape[0] == y.shape[0], "Features and labels must have the same number of samples"
        self.x = x
        self.y = y

    def __len__(self):
        """
        Total number of samples.
        """
        return self.x.shape[0]

    def __getitem__(self, index):
        """
        Generate one sample of data.
        """
        # Convert the data to tensors when requested
        return torch.from_numpy(self.x[index]).float().to(device), torch.from_numpy(self.y[index]).float().to(device)

x_test = hkl.load(DATA_PATH+'x_test_v1_1e-15.hkl')
x_valid = hkl.load(DATA_PATH+'c_data_x_8_1_v1_1e-15.hkl')
y_valid = hkl.load(DATA_PATH+'c_data_y_8_1_v1_1e-15.hkl')


def gen_pe(max_length, d_model, n):

  # generate an empty matrix for the positional encodings (pe)
  pe = np.zeros(max_length*d_model).reshape(max_length, d_model) 

  # for each position
  for k in np.arange(max_length):

    # for each dimension
    for i in np.arange(d_model//2):

      # calculate the internal value for sin and cos
      theta = k / (n ** ((2*i)/d_model))       

      # even dims: sin   
      pe[k, 2*i] = math.sin(theta) 

      # odd dims: cos               
      pe[k, 2*i+1] = math.cos(theta)

  return pe


val_dataset = NumpyDataset(x_valid, y_valid)

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [13]:
device

device(type='cuda')

In [14]:

def format_time(elapsed):
    """Take a time in seconds and return a string hh:mm:ss."""
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

ts = time.time()

gc.collect()

print("Time after processing data:", format_time(time.time()-ts), flush=True)    


gc.collect()



class FFNN_Transformer_1_Renorm(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_Transformer_1_Renorm, self).__init__()
        
        self.encode_dim = 180
        self.hidden_dim = 180
        self.iter_dim = 1024
        
        self.pe = gen_pe(max_length=60, d_model= self.encode_dim, n=100)
        self.pe = torch.from_numpy((self.pe).astype(np.float32)).to(device)[None,:,:]
        self.encoder_layer = TransformerEncoderLayer(d_model=self.encode_dim, nhead=6,dim_feedforward=512,dropout=0, batch_first=True)
        self.transformer_encoder = TransformerEncoder(self.encoder_layer, num_layers=5)  
        # self.LSTM_1 = LSTM(self.encode_dim,self.hidden_dim,6,batch_first=True,dropout=0.05,bidirectional=True)
        self.input_size = input_size
        self.pos_emb_size = 20
        self.pos_emb = nn.Linear(self.pos_emb_size,60)
        self.Linear_1 = nn.Linear(self.pos_emb_size+len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_1_0 = nn.Linear(self.encode_dim, self.encode_dim)

        self.Linear_2 = nn.Linear(3*self.hidden_dim+self.encode_dim, self.iter_dim)
        self.Linear_2_0 = nn.Linear(self.iter_dim, self.iter_dim)
        self.Linear_2_1 = nn.Linear(self.iter_dim, self.iter_dim)
        self.Linear_2_2 = nn.Linear(self.iter_dim, self.iter_dim)

        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        self.Linear_4 = nn.Linear(len(seq_fea_list)*60+len(num_fea_list)+60*self.iter_dim+len(num_fea_list), 2*self.iter_dim)
        self.Linear_5 = nn.Linear(2*self.iter_dim,len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)

        self.fea_weight = nn.Linear(len(seq_fea_list)*60,1)
        self.fea_bias = nn.Linear(len(seq_fea_list)*60,1)
        self.bn1 = BatchNorm1d(self.iter_dim)
        self.bn2 = BatchNorm1d(self.iter_dim)
        self.bn3 = BatchNorm1d(self.iter_dim)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        x_seq = x_seq*self.fea_weight.weight[None,:,:]+self.fea_bias.weight[None,:,:]
        
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        x_pos_emb = self.pos_emb.weight.reshape(1,60,self.pos_emb_size)
        x_pos_emb = x_pos_emb.repeat((x.shape[0],1,1))
        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat,x_pos_emb),dim=-1)/3))
        
        
        x_seq_1 = self.transformer_encoder(self.Linear_1_0(x_seq)/3+self.pe)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))

        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/8))
        x_seq_1 = F.elu(self.Linear_2_0(x_seq_1)) + x_seq_1
        # x_seq_1 = self.bn1(x_seq_1)
        x_seq_1 = F.elu(self.Linear_2_1(x_seq_1)) + x_seq_1
        # x_seq_1 = self.bn2(x_seq_1)
        x_seq_1 = F.elu(self.Linear_2_2(x_seq_1)) + x_seq_1
        # x_seq_1 = self.bn3(x_seq_1)
        x_seq_out = self.Linear_3(x_seq_1/3)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_seq_out_1 = x_seq_1.reshape((-1,60*self.iter_dim))
        x_num_out = F.elu(self.Linear_4(torch.concat((x_seq_out_1,x_num,x),dim=-1)/20))
        x_num_out = self.Linear_5(x_num_out/4)
        
        # print(self.weight)
        
        return self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3




class FFNN_LSTM(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_LSTM, self).__init__()
        
        self.encode_dim = 120
        self.hidden_dim = 120
        self.iter_dim = 512

        
        self.LSTM_1 = LSTM(self.encode_dim,self.hidden_dim,2,batch_first=True,dropout=0.05,bidirectional=True)
        self.input_size = input_size
        self.Linear_1 = nn.Linear(len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(2*self.hidden_dim+self.encode_dim, self.iter_dim)
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        self.Linear_4 = nn.Linear(60*self.iter_dim+len(num_fea_list), 2*self.iter_dim)
        self.Linear_5 = nn.Linear(2*self.iter_dim,len(num_y_list))


    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/20))
        
        x_seq_1,_ = self.LSTM_1(x_seq/20)
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq),dim=-1)/20))
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_seq_out_1 = x_seq_1.reshape((-1,60*self.iter_dim))
        x_num_out = F.elu(self.Linear_4(torch.concat((x_seq_out_1,x_num),dim=-1)/20))
        x_num_out = self.Linear_5(x_num_out/20)
        
        return torch.concat((x_seq_out,x_num_out),dim=-1)

class FFNN_LSTM_7_AVG_ATT(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_LSTM_7_AVG_ATT, self).__init__()
        
        self.encode_dim = 256
        self.hidden_dim = 280
        self.iter_dim = 1024

        
        self.LSTM_1 = LSTM(self.encode_dim,self.hidden_dim,6,batch_first=True,dropout=0.05,bidirectional=True)
        self.input_size = input_size
        self.Linear_1 = nn.Linear(len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(6*self.hidden_dim+self.encode_dim, self.iter_dim)
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        self.Linear_3_0 = nn.Linear(self.iter_dim, 1)

        self.Linear_4_0 = nn.Linear(self.iter_dim, self.iter_dim*2)

        self.Linear_4 = nn.Linear(self.iter_dim*2, len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        # print(x_seq)
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))
        # print(x_seq)
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        # print(x_num_repeat.shape)
        # print(x_seq.shape)

        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/5))
        
        x_seq_1,_ = self.LSTM_1(x_seq/5)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))

        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/5))
        
        att_weight = F.softmax(self.Linear_3_0(x_seq_1) - 10,dim=1)
        
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_num_out = F.elu(self.Linear_4_0(torch.sum(att_weight*x_seq_1,dim=1)))
        x_num_out = self.Linear_4(x_num_out)
        
        # print(self.weight)
        
        return self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3


class FFNN_LSTM_749_AVG_ATT(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_LSTM_749_AVG_ATT, self).__init__()
        
        self.encode_dim = 256
        self.hidden_dim = 320
        self.iter_dim = 1024

        
        self.LSTM_1 = LSTM(self.encode_dim,self.hidden_dim,6,batch_first=True,dropout=0.05,bidirectional=True)
        self.input_size = input_size
        self.Linear_1 = nn.Linear(len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(6*self.hidden_dim+self.encode_dim, self.iter_dim)
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        self.Linear_3_0 = nn.Linear(self.iter_dim, 1)

        self.Linear_4_0 = nn.Linear(self.iter_dim, self.iter_dim*2)

        self.Linear_4 = nn.Linear(self.iter_dim*2, len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        # print(x_seq)
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))
        # print(x_seq)
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        # print(x_num_repeat.shape)
        # print(x_seq.shape)

        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/5))
        
        x_seq_1,_ = self.LSTM_1(x_seq/5)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))

        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/5))
        
        att_weight = F.softmax(self.Linear_3_0(x_seq_1) - 10,dim=1)
        
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_num_out = F.elu(self.Linear_4_0(torch.sum(att_weight*x_seq_1,dim=1)))
        x_num_out = self.Linear_4(x_num_out)
        
        # print(self.weight)
        
        return self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3



class FFNN_LSTM_888_AVG_ATT(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_LSTM_888_AVG_ATT, self).__init__()
        
        self.encode_dim = 256
        self.hidden_dim = 256
        self.iter_dim = 800

        
        self.LSTM_1 = LSTM(self.encode_dim,self.hidden_dim,6,batch_first=True,dropout=0.01,bidirectional=True)
        self.input_size = input_size
        self.Linear_1 = nn.Linear(len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(6*self.hidden_dim+self.encode_dim, self.iter_dim)
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        self.Linear_3_0 = nn.Linear(self.iter_dim, 1)

        self.Linear_4_0 = nn.Linear(self.iter_dim, self.iter_dim*2)

        self.Linear_4 = nn.Linear(self.iter_dim*2, len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        # print(x_seq)
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))
        # print(x_seq)
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        # print(x_num_repeat.shape)
        # print(x_seq.shape)

        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/5))
        
        x_seq_1,_ = self.LSTM_1(x_seq/5)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))

        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/5))
        
        att_weight = F.softmax(self.Linear_3_0(x_seq_1) - 10,dim=1)
        
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_num_out = F.elu(self.Linear_4_0(torch.sum(att_weight*x_seq_1,dim=1)))
        x_num_out = self.Linear_4(x_num_out)
        
        # print(self.weight)
        
        return self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3


class FFNN_LSTM_999_AVG_ATT(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_LSTM_999_AVG_ATT, self).__init__()
        
        self.encode_dim = 256
        self.hidden_dim = 300
        self.iter_dim = 800

        
        self.LSTM_1 = LSTM(self.encode_dim,self.hidden_dim,7,batch_first=True,dropout=0.01,bidirectional=True)
        self.input_size = input_size
        self.Linear_1 = nn.Linear(len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(6*self.hidden_dim+self.encode_dim, self.iter_dim)
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        self.Linear_3_0 = nn.Linear(self.iter_dim, 1)

        self.Linear_4_0 = nn.Linear(self.iter_dim, self.iter_dim*2)

        self.Linear_4 = nn.Linear(self.iter_dim*2, len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        # print(x_seq)
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))
        # print(x_seq)
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        # print(x_num_repeat.shape)
        # print(x_seq.shape)

        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/5))
        
        x_seq_1,_ = self.LSTM_1(x_seq/5)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))

        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/5))
        
        att_weight = F.softmax(self.Linear_3_0(x_seq_1) - 10,dim=1)
        
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_num_out = F.elu(self.Linear_4_0(torch.sum(att_weight*x_seq_1,dim=1)))
        x_num_out = self.Linear_4(x_num_out)
        
        # print(self.weight)
        
        return self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3





class FFNN_LSTM_4_AVG_ATT(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_LSTM_4_AVG_ATT, self).__init__()
        
        self.encode_dim = 380
        self.hidden_dim = 380
        self.iter_dim = 1024

        
        self.LSTM_1 = LSTM(self.encode_dim,self.hidden_dim,4,batch_first=True,dropout=0.05,bidirectional=True)
        self.input_size = input_size
        self.Linear_1 = nn.Linear(len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(6*self.hidden_dim+self.encode_dim, self.iter_dim)
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        self.Linear_3_0 = nn.Linear(self.iter_dim, 1)

        self.Linear_4_0 = nn.Linear(self.iter_dim, self.iter_dim*2)

        self.Linear_4 = nn.Linear(self.iter_dim*2, len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        # print(x_seq)
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))
        # print(x_seq)
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        # print(x_num_repeat.shape)
        # print(x_seq.shape)

        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/5))
        
        x_seq_1,_ = self.LSTM_1(x_seq/5)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))

        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/5))
        
        att_weight = F.softmax(self.Linear_3_0(x_seq_1) - 10,dim=1)
        
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_num_out = F.elu(self.Linear_4_0(torch.sum(att_weight*x_seq_1,dim=1)))
        x_num_out = self.Linear_4(x_num_out)
        
        # print(self.weight)
        
        return self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3




class FFNN_UNET_4(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_UNET_4, self).__init__()
        
        self.encode_dim = 150
        self.hidden_dim = 220
        self.iter_dim = 450
        

        self.enc_channels= (self.encode_dim, self.encode_dim*2, self.encode_dim*4, self.encode_dim*8)
        self.dec_channels= (self.encode_dim*8, self.encode_dim*4, self.encode_dim*2)

        self.encoder = Encoder(self.enc_channels)
        self.decoder = Decoder(self.dec_channels)
        
        # self.LSTM_1 = LSTM(self.encode_dim,self.hidden_dim,5,batch_first=True,dropout=0.05,bidirectional=True)
        self.input_size = input_size
        self.Linear_1 = nn.Linear(len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(6*self.encode_dim+self.encode_dim, self.iter_dim)
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        
        self.Linear_4_0 = nn.Linear(self.iter_dim, self.iter_dim*2)

        self.Linear_4 = nn.Linear(self.iter_dim*2, len(num_y_list))
   
        
        
        # self.Linear_4 = nn.Linear(len(seq_fea_list)*60+len(num_fea_list)+60*self.iter_dim+len(num_fea_list), 2*self.iter_dim)
        self.Linear_5 = nn.Linear(2*self.iter_dim,len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/5))
        
        enc_ftrs = self.encoder(torch.transpose(x_seq,1, 2))
        x_seq_1 = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
        x_seq_1 = torch.transpose(x_seq_1, 1, 2)
        
        # print(x_seq_1.shape)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))

        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/5))
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_num_out = F.elu(self.Linear_4_0(torch.mean(x_seq_1,dim=1)))
        x_num_out = self.Linear_4(x_num_out)

        output = self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3

        output[:,zeroout_index] =  output[:,zeroout_index]*0.0

        
        
        # print(self.weight)
        
        return output
    
class FFNN_LSTM_6_AVG(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_LSTM_6_AVG, self).__init__()
        
        self.encode_dim = 300
        self.hidden_dim = 280
        self.iter_dim = 800

        
        self.LSTM_1 = LSTM(self.encode_dim,self.hidden_dim,6,batch_first=True,dropout=0.01,bidirectional=True)
        self.input_size = input_size
        self.Linear_1 = nn.Linear(len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(6*self.hidden_dim+self.encode_dim, self.iter_dim)
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        self.Linear_4_0 = nn.Linear(self.iter_dim, self.iter_dim*2)

        self.Linear_4 = nn.Linear(self.iter_dim*2, len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/5))
        
        x_seq_1,_ = self.LSTM_1(x_seq/5)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))

        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/5))
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_num_out = F.elu(self.Linear_4_0(torch.mean(x_seq_1,dim=1)))
        x_num_out = self.Linear_4(x_num_out)

        output = self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3
        
        output[:,zeroout_index] =  output[:,zeroout_index]*0.0
        # print(self.weight)
        
        return output

 
class FFNN_LSTM_6_AVG_4(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_LSTM_6_AVG_4, self).__init__()
        
        self.encode_dim = 300
        self.hidden_dim = 400
        self.iter_dim = 1400

        
        self.LSTM_1 = LSTM(self.encode_dim,self.hidden_dim,6,batch_first=True,dropout=0.05,bidirectional=True)
        self.input_size = input_size
        self.Linear_1 = nn.Linear(len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(6*self.hidden_dim+self.encode_dim, self.iter_dim)
        
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        self.Linear_3_0 = nn.Linear(self.iter_dim, self.iter_dim)
        self.Linear_4_0 = nn.Linear(self.iter_dim, self.iter_dim*2)

        self.Linear_4 = nn.Linear(self.iter_dim*2, len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/5))
        
        x_seq_1,_ = self.LSTM_1(x_seq/5)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))

        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/5))
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = self.Linear_3_0(F.elu(x_seq_out)) + x_seq_out
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_num_out = F.elu(self.Linear_4_0(torch.mean(x_seq_1,dim=1)))
        x_num_out = self.Linear_4(x_num_out)

        output = self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3
        
        output[:,zeroout_index] =  output[:,zeroout_index]*0.0
        # print(self.weight)
        
        return output

class FFNN_LSTM_6_AVG_1(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_LSTM_6_AVG_1, self).__init__()
        
        self.encode_dim = 300
        self.hidden_dim = 300
        self.iter_dim = 1400

        
        self.LSTM_a = LSTM(self.encode_dim,self.hidden_dim,6,batch_first=True,dropout=0.05,bidirectional=True)

        self.Seq_0_Linear_1 = nn.Linear(1, self.encode_dim)
        self.Seq_0_Linear_2 = nn.Linear(self.encode_dim,1)

        self.Seq_1_Linear_1 = nn.Linear(1, self.encode_dim)
        self.Seq_1_Linear_2 = nn.Linear(self.encode_dim,1)

        self.Seq_2_Linear_1 = nn.Linear(1, self.encode_dim)
        self.Seq_2_Linear_2 = nn.Linear(self.encode_dim,1)

        self.Seq_3_Linear_1 = nn.Linear(1, self.encode_dim)
        self.Seq_3_Linear_2 = nn.Linear(self.encode_dim,1)

        self.Seq_4_Linear_1 = nn.Linear(1, self.encode_dim)
        self.Seq_4_Linear_2 = nn.Linear(self.encode_dim,1)

        self.Seq_5_Linear_1 = nn.Linear(1, self.encode_dim)
        self.Seq_5_Linear_2 = nn.Linear(self.encode_dim,1)
        
        self.input_size = input_size
        self.Linear_1 = nn.Linear(len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(6*self.hidden_dim+self.encode_dim, self.iter_dim)
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        self.Linear_4_0 = nn.Linear(self.iter_dim, self.iter_dim*2)

        self.Linear_4 = nn.Linear(self.iter_dim*2, len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))

        x_seq_1d_out = x_seq*0

        x_seq_1d_out[:,:,0] = self.Seq_0_Linear_2(F.elu(self.Seq_0_Linear_1(x_seq[:,:,0][:,:,None])))[:,:,0]
        x_seq_1d_out[:,:,1] = self.Seq_1_Linear_2(F.elu(self.Seq_1_Linear_1(x_seq[:,:,1][:,:,None])))[:,:,0]
        x_seq_1d_out[:,:,2] = self.Seq_2_Linear_2(F.elu(self.Seq_2_Linear_1(x_seq[:,:,2][:,:,None])))[:,:,0]
        x_seq_1d_out[:,:,3] = self.Seq_3_Linear_2(F.elu(self.Seq_3_Linear_1(x_seq[:,:,3][:,:,None])))[:,:,0]
        x_seq_1d_out[:,:,4] = self.Seq_4_Linear_2(F.elu(self.Seq_4_Linear_1(x_seq[:,:,4][:,:,None])))[:,:,0]
        x_seq_1d_out[:,:,5] = self.Seq_5_Linear_2(F.elu(self.Seq_5_Linear_1(x_seq[:,:,5][:,:,None])))[:,:,0]

        x_seq_1d_out = x_seq_1d_out.reshape((-1,len(seq_fea_list)*60))
        
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/5))
        
        x_seq_1,_ = self.LSTM_a(x_seq/5)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))

        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/5))
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = 0.0000001*x_seq_out.reshape((-1,60*len(seq_y_list))) + x_seq_1d_out
        
        x_num_out = F.elu(self.Linear_4_0(torch.mean(x_seq_1,dim=1)))
        x_num_out = self.Linear_4(x_num_out)

        output = self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3
        
        output[:,zeroout_index] =  output[:,zeroout_index]*0.0
        # print(self.weight)
        
        return output


class FFNN_LSTM_6_AVG_2(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_LSTM_6_AVG_2, self).__init__()
        
        self.encode_dim = 300
        self.hidden_dim = 300
        self.iter_dim = 1400

        self.small_hidden_dim = 30
        
        self.LSTM_a = LSTM(self.encode_dim,self.hidden_dim,6,batch_first=True,dropout=0.05,bidirectional=True)

        self.LSTM_0 = LSTM(1,self.small_hidden_dim,3,batch_first=True,dropout=0.05,bidirectional=True)
        self.LSTM_1 = LSTM(1,self.small_hidden_dim,3,batch_first=True,dropout=0.05,bidirectional=True)
        self.LSTM_2 = LSTM(1,self.small_hidden_dim,3,batch_first=True,dropout=0.05,bidirectional=True)
        self.LSTM_3 = LSTM(1,self.small_hidden_dim,3,batch_first=True,dropout=0.05,bidirectional=True)
        self.LSTM_4 = LSTM(1,self.small_hidden_dim,3,batch_first=True,dropout=0.05,bidirectional=True)
        self.LSTM_5 = LSTM(1,self.small_hidden_dim,3,batch_first=True,dropout=0.05,bidirectional=True)

        self.LSTM_0_Linear = nn.Linear(self.small_hidden_dim*2, 1)
        self.LSTM_1_Linear = nn.Linear(self.small_hidden_dim*2, 1)
        self.LSTM_2_Linear = nn.Linear(self.small_hidden_dim*2, 1)
        self.LSTM_3_Linear = nn.Linear(self.small_hidden_dim*2, 1)
        self.LSTM_4_Linear = nn.Linear(self.small_hidden_dim*2, 1)
        self.LSTM_5_Linear = nn.Linear(self.small_hidden_dim*2, 1)
        
        self.Seq_0_Linear_1 = nn.Linear(1, self.encode_dim)
        self.Seq_0_Linear_2 = nn.Linear(self.encode_dim,1)

        self.Seq_1_Linear_1 = nn.Linear(1, self.encode_dim)
        self.Seq_1_Linear_2 = nn.Linear(self.encode_dim,1)

        self.Seq_2_Linear_1 = nn.Linear(1, self.encode_dim)
        self.Seq_2_Linear_2 = nn.Linear(self.encode_dim,1)

        self.Seq_3_Linear_1 = nn.Linear(1, self.encode_dim)
        self.Seq_3_Linear_2 = nn.Linear(self.encode_dim,1)

        self.Seq_4_Linear_1 = nn.Linear(1, self.encode_dim)
        self.Seq_4_Linear_2 = nn.Linear(self.encode_dim,1)

        self.Seq_5_Linear_1 = nn.Linear(1, self.encode_dim)
        self.Seq_5_Linear_2 = nn.Linear(self.encode_dim,1)
        
        self.input_size = input_size
        self.Linear_1 = nn.Linear(len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(6*self.hidden_dim+self.encode_dim, self.iter_dim)
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        self.Linear_4_0 = nn.Linear(self.iter_dim, self.iter_dim*2)

        self.Linear_4 = nn.Linear(self.iter_dim*2, len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))

        x_seq_1d_out = x_seq[:,0:6,:]*0
        x_seq_1d_out = torch.transpose(x_seq_1d_out, 1, 2)

        tmp_0 = self.Seq_0_Linear_2(F.elu(self.Seq_0_Linear_1(x_seq_1d_out[:,:,0][:,:,None])))[:,:,0]
        tmp_1 = self.Seq_1_Linear_2(F.elu(self.Seq_1_Linear_1(x_seq_1d_out[:,:,1][:,:,None])))[:,:,0]
        tmp_2 = self.Seq_2_Linear_2(F.elu(self.Seq_2_Linear_1(x_seq_1d_out[:,:,2][:,:,None])))[:,:,0]
        tmp_3 = self.Seq_3_Linear_2(F.elu(self.Seq_3_Linear_1(x_seq_1d_out[:,:,3][:,:,None])))[:,:,0]
        tmp_4 = self.Seq_4_Linear_2(F.elu(self.Seq_4_Linear_1(x_seq_1d_out[:,:,4][:,:,None])))[:,:,0]
        tmp_5 = self.Seq_5_Linear_2(F.elu(self.Seq_5_Linear_1(x_seq_1d_out[:,:,5][:,:,None])))[:,:,0]

        x_seq_1d_out = torch.cat((tmp_0[:,:,None],tmp_1[:,:,None],tmp_2[:,:,None],tmp_3[:,:,None],tmp_4[:,:,None],tmp_5[:,:,None]),dim=-1)

        
        x_seq_1d_out = torch.transpose(x_seq_1d_out, 1, 2)
        x_seq_1d_out = x_seq_1d_out.reshape((-1,len(seq_y_list)*60))
        
        x_seq_2d_out = x_seq[:,0:6,:]*0
        x_seq_2d_out = torch.transpose(x_seq_2d_out, 1, 2)
        
        tmp_0 = self.LSTM_0_Linear(self.LSTM_0(x_seq_2d_out[:,:,0][:,:,None])[0])[:,:,0]
        tmp_1 = self.LSTM_1_Linear(self.LSTM_1(x_seq_2d_out[:,:,1][:,:,None])[0])[:,:,0]
        tmp_2 = self.LSTM_2_Linear(self.LSTM_2(x_seq_2d_out[:,:,2][:,:,None])[0])[:,:,0]
        tmp_3 = self.LSTM_3_Linear(self.LSTM_3(x_seq_2d_out[:,:,3][:,:,None])[0])[:,:,0]
        tmp_4 = self.LSTM_4_Linear(self.LSTM_4(x_seq_2d_out[:,:,4][:,:,None])[0])[:,:,0]
        tmp_5 = self.LSTM_5_Linear(self.LSTM_5(x_seq_2d_out[:,:,5][:,:,None])[0])[:,:,0]

        x_seq_2d_out = torch.cat((tmp_0[:,:,None],tmp_1[:,:,None],tmp_2[:,:,None],tmp_3[:,:,None],tmp_4[:,:,None],tmp_5[:,:,None]),dim=-1)

        x_seq_2d_out = torch.transpose(x_seq_2d_out, 1, 2)
        x_seq_2d_out = x_seq_2d_out.reshape((-1,len(seq_y_list)*60))

        #print(x_seq_1d_out.shape)
        #print(x_seq_2d_out.shape)

        
        x_seq = torch.transpose(x_seq, 1, 2)

        
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/5))
        
        x_seq_1,_ = self.LSTM_a(x_seq/5)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))


        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/5))
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = (x_seq_out.reshape((-1,60*len(seq_y_list)))) + x_seq_1d_out +  (x_seq_2d_out)
        
        x_num_out = F.elu(self.Linear_4_0(torch.mean(x_seq_1,dim=1)))
        x_num_out = self.Linear_4(x_num_out)

        output = self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3
        
        output[:,zeroout_index] =  output[:,zeroout_index]*0.0
        # print(self.weight)
        
        return output







class FFNN_GRU_6_AVG(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_GRU_6_AVG, self).__init__()
        
        self.encode_dim = 290
        self.hidden_dim = 280
        self.iter_dim = 1024

        
        self.LSTM_1 = GRU(self.encode_dim,self.hidden_dim,3,batch_first=True,dropout=0.05,bidirectional=True)
        self.input_size = input_size
        self.Linear_1 = nn.Linear(len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(6*self.hidden_dim+self.encode_dim, self.iter_dim)
        self.Linear_2_0 = nn.Linear(self.iter_dim, self.iter_dim)
        self.Linear_2_1 = nn.Linear(self.iter_dim, self.iter_dim)


        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        self.Linear_4_0 = nn.Linear(self.iter_dim, self.iter_dim*2)

        self.Linear_4 = nn.Linear(self.iter_dim*2, len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/5))
        
        x_seq_1,_ = self.LSTM_1(x_seq/5)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))

        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/5))
        x_seq_1 = x_seq_1 + 0.01*self.Linear_2_1(F.elu(self.Linear_2_0(x_seq_1)))
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_num_out = F.elu(self.Linear_4_0(torch.mean(x_seq_1,dim=1)))
        x_num_out = self.Linear_4(x_num_out)
        
        # print(self.weight)
        
        return self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3
         


class FFNN_LSTM_7(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_LSTM_7, self).__init__()
        
        self.encode_dim = 160
        self.hidden_dim = 250
        self.iter_dim = 400

        
        self.LSTM_1 = LSTM(self.encode_dim,self.hidden_dim,7,batch_first=True,dropout=0.05,bidirectional=True)
        self.input_size = input_size
        self.Linear_1 = nn.Linear(len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(6*self.hidden_dim+self.encode_dim, self.iter_dim)
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        self.Linear_4 = nn.Linear(len(seq_fea_list)*60+len(num_fea_list)+60*self.iter_dim+len(num_fea_list), 2*self.iter_dim)
        self.Linear_5 = nn.Linear(2*self.iter_dim,len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))
        print(x_seq.shape)
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/5))
        
        x_seq_1,_ = self.LSTM_1(x_seq/5)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))

        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/5))
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_seq_out_1 = x_seq_1.reshape((-1,60*self.iter_dim))
        x_num_out = F.elu(self.Linear_4(torch.concat((x_seq_out_1,x_num,x),dim=-1)/5))
        x_num_out = self.Linear_5(x_num_out/5)
        
        # print(self.weight)
        
        return self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3
       


class Conv1dBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_channels), 
            nn.ReLU(inplace=True),
            nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        return self.layers(x)

class Encoder(nn.Module):
    def __init__(self, channels=(1, 16, 32, 64)):
        super().__init__()
        self.enc_blocks = nn.ModuleList([Conv1dBlock(channels[i], channels[i+1]) 
                                         for i in range(len(channels)-1)])
        self.pool = nn.AvgPool1d(2,2)
    
    def forward(self, x):
        features = []
        for block in self.enc_blocks:
            x = block(x)
            features.append(x)
            x = self.pool(x)
        return features

class Decoder(nn.Module):
    def __init__(self, channels=(64, 32, 16)):
        super().__init__()
        self.channels = channels
        self.upconvs = nn.ModuleList([nn.ConvTranspose1d(channels[i], channels[i+1], 2, 2) 
                                      for i in range(len(channels)-1)])
        self.dec_blocks = nn.ModuleList([Conv1dBlock(channels[i], channels[i+1]) 
                                         for i in range(len(channels)-1)])
        
    def forward(self, x, encoder_features):
        for i in range(len(self.channels)-1):
            x = self.upconvs[i](x)
            enc_ftrs = self.crop(encoder_features[i], x)
            x = torch.cat([x, enc_ftrs], dim=1)
            x = self.dec_blocks[i](x)
        return x
    
    def crop(self, enc_ftrs, x):
        _, _, T = x.shape
        enc_ftrs = enc_ftrs[:, :, (enc_ftrs.shape[2] - T) // 2 : (enc_ftrs.shape[2] - T) // 2 + T]
        return enc_ftrs

class UNet1D(nn.Module):
    def __init__(self, enc_channels=(1, 16, 32, 64), dec_channels=(64, 32, 16), nb_classes=1):
        super().__init__()
        self.encoder = Encoder(enc_channels)
        self.decoder = Decoder(dec_channels)

    def forward(self, x):
        enc_ftrs = self.encoder(x)
        out = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
        return out
    


class FFNN_UNET(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_UNET, self).__init__()
        
        self.encode_dim = 300
        self.hidden_dim = 180
        self.iter_dim = 300
        

        self.enc_channels= (self.encode_dim, self.encode_dim*2, self.encode_dim*4, self.encode_dim*8)
        self.dec_channels= (self.encode_dim*8, self.encode_dim*4, self.encode_dim*2)

        self.encoder = Encoder(self.enc_channels)
        self.decoder = Decoder(self.dec_channels)
        
        # self.LSTM_1 = LSTM(self.encode_dim,self.hidden_dim,5,batch_first=True,dropout=0.05,bidirectional=True)
        self.input_size = input_size
        self.Linear_1 = nn.Linear(len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(6*self.encode_dim+self.encode_dim, self.iter_dim)
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        self.Linear_4 = nn.Linear(len(seq_fea_list)*60+len(num_fea_list)+60*self.iter_dim+len(num_fea_list), 2*self.iter_dim)
        self.Linear_5 = nn.Linear(2*self.iter_dim,len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/5))
        
        enc_ftrs = self.encoder(torch.transpose(x_seq,1, 2))
        x_seq_1 = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
        x_seq_1 = torch.transpose(x_seq_1, 1, 2)
        
        # print(x_seq_1.shape)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))

        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/5))
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_seq_out_1 = x_seq_1.reshape((-1,60*self.iter_dim))
        x_num_out = F.elu(self.Linear_4(torch.concat((x_seq_out_1,x_num,x),dim=-1)/5))
        x_num_out = self.Linear_5(x_num_out/5)
        
        # print(self.weight)
        
        return self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3
       



class FFNN_Transformer_2(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_Transformer_2, self).__init__()
        
        self.encode_dim = 200
        self.hidden_dim = self.encode_dim
        self.iter_dim = self.encode_dim
        self.encoder_layer_1 = TransformerEncoderLayer(d_model=self.encode_dim, nhead=5,dim_feedforward=512,activation='gelu',batch_first=True)
        self.transformer_encoder_1 = TransformerEncoder(self.encoder_layer_1, num_layers=6)  
        
        self.encoder_layer_2 = TransformerEncoderLayer(d_model=self.encode_dim, nhead=5,dim_feedforward=512,activation='gelu',batch_first=True)
        self.transformer_encoder_2 = TransformerEncoder(self.encoder_layer_2, num_layers=3) 
        self.encoder_layer_unfold_2 = nn.Linear(self.encode_dim,60)
        
        
        self.input_size = input_size
        self.pos_emb_size = 8
        self.pos_emb = nn.Linear(self.pos_emb_size,60)
        self.Linear_1 = nn.Linear(self.pos_emb_size+len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_1_otherside = nn.Linear(60, self.encode_dim)
        
        self.Linear_2 = nn.Linear(2*self.hidden_dim+self.encode_dim, self.iter_dim)
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        self.Linear_4 = nn.Linear(len(seq_fea_list)*60+len(num_fea_list)+60*self.iter_dim+len(num_fea_list), 2*self.iter_dim)
        self.Linear_5 = nn.Linear(2*self.iter_dim,len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.pos_emb_size = 8
        self.pos_emb = nn.Linear(self.pos_emb_size,60)

        
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]     
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))
        
        x_seq_otherside = self.encoder_layer_unfold_2(self.transformer_encoder_2(self.Linear_1_otherside(x_seq)))
        
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        x_pos_emb = self.pos_emb.weight.reshape(1,60,self.pos_emb_size)
        x_pos_emb = x_pos_emb.repeat((x.shape[0],1,1))
        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat,x_pos_emb),dim=-1)/4))
        
        x_seq_1 = self.transformer_encoder_1(x_seq)
        
        
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq),dim=-1)/4))
        
        x_seq_out = self.Linear_3(x_seq_1) + torch.transpose(x_seq_otherside[:,0:6,:],1,2)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_seq_out_1 = x_seq_1.reshape((-1,60*self.iter_dim))
        x_num_out = F.elu(self.Linear_4(torch.concat((x_seq_out_1,x_num,x),dim=-1)/5))
        x_num_out = self.Linear_5(x_num_out/6)
        
        # print(self.weight)
        
        return self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3
       

class FFNN_UNET_3(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_UNET_3, self).__init__()
        
        self.encode_dim = 259
        self.hidden_dim = 200
        self.iter_dim = 320
        

        self.enc_channels= (self.encode_dim, self.encode_dim*2, self.encode_dim*4, self.encode_dim*8)
        self.dec_channels= (self.encode_dim*8, self.encode_dim*4, self.encode_dim*2)

        self.encoder = Encoder(self.enc_channels)
        self.decoder = Decoder(self.dec_channels)
        
        # self.LSTM_1 = LSTM(self.encode_dim,self.hidden_dim,5,batch_first=True,dropout=0.05,bidirectional=True)
        self.input_size = input_size
        self.Linear_1 = nn.Linear(len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(6*self.encode_dim+self.encode_dim, self.iter_dim)
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        
        self.Linear_4_0 = nn.Linear(self.iter_dim, self.iter_dim*2)

        self.Linear_4 = nn.Linear(self.iter_dim*2, len(num_y_list))
   
        
        
        # self.Linear_4 = nn.Linear(len(seq_fea_list)*60+len(num_fea_list)+60*self.iter_dim+len(num_fea_list), 2*self.iter_dim)
        self.Linear_5 = nn.Linear(2*self.iter_dim,len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/5))
        
        enc_ftrs = self.encoder(torch.transpose(x_seq,1, 2))
        x_seq_1 = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
        x_seq_1 = torch.transpose(x_seq_1, 1, 2)
        
        # print(x_seq_1.shape)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))

        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/5))
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_num_out = F.elu(self.Linear_4_0(torch.mean(x_seq_1,dim=1)))
        x_num_out = self.Linear_4(x_num_out)
        
        
        # print(self.weight)
        
        return self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3
    

class FFNN_UNET_5(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_UNET_5, self).__init__()
        
        self.encode_dim = 310
        self.hidden_dim = 200
        self.iter_dim = 320
        

        self.enc_channels= (self.encode_dim, self.encode_dim*2, self.encode_dim*4, self.encode_dim*8)
        self.dec_channels= (self.encode_dim*8, self.encode_dim*4, self.encode_dim*2)

        self.encoder = Encoder(self.enc_channels)
        self.decoder = Decoder(self.dec_channels)
        
        # self.LSTM_1 = LSTM(self.encode_dim,self.hidden_dim,5,batch_first=True,dropout=0.05,bidirectional=True)
        self.input_size = input_size
        self.Linear_1 = nn.Linear(len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(6*self.encode_dim+self.encode_dim, self.iter_dim)
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        
        self.Linear_4_0 = nn.Linear(self.iter_dim, self.iter_dim*2)

        self.Linear_4 = nn.Linear(self.iter_dim*2, len(num_y_list))
   
        
        
        # self.Linear_4 = nn.Linear(len(seq_fea_list)*60+len(num_fea_list)+60*self.iter_dim+len(num_fea_list), 2*self.iter_dim)
        self.Linear_5 = nn.Linear(2*self.iter_dim,len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/5))
        
        enc_ftrs = self.encoder(torch.transpose(x_seq,1, 2))
        x_seq_1 = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
        x_seq_1 = torch.transpose(x_seq_1, 1, 2)
        
        # print(x_seq_1.shape)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))

        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/5))
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_num_out = F.elu(self.Linear_4_0(torch.mean(x_seq_1,dim=1)))
        x_num_out = self.Linear_4(x_num_out)
        
        
        # print(self.weight)
        
        return self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3




class FFNN_UNET_6(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNN_UNET_6, self).__init__()
        
        self.encode_dim = 200
        self.hidden_dim = 160
        self.iter_dim = 500
        

        self.enc_channels= (self.encode_dim, self.encode_dim*2, self.encode_dim*4, self.encode_dim*8)
        self.dec_channels= (self.encode_dim*8, self.encode_dim*4, self.encode_dim*2)

        self.encoder = Encoder(self.enc_channels)
        self.decoder = Decoder(self.dec_channels)
        
        # self.LSTM_1 = LSTM(self.encode_dim,self.hidden_dim,5,batch_first=True,dropout=0.05,bidirectional=True)
        self.input_size = input_size
        self.Linear_1 = nn.Linear(len(seq_fea_list)+len(num_fea_list), self.encode_dim)
        self.Linear_2 = nn.Linear(6*self.encode_dim+self.encode_dim, self.iter_dim)
        self.Linear_3 = nn.Linear(self.iter_dim, len(seq_y_list))
        
        self.Linear_4_0 = nn.Linear(self.iter_dim, self.iter_dim*2)

        self.Linear_4 = nn.Linear(self.iter_dim*2, len(num_y_list))
   
        
        
        # self.Linear_4 = nn.Linear(len(seq_fea_list)*60+len(num_fea_list)+60*self.iter_dim+len(num_fea_list), 2*self.iter_dim)
        self.Linear_5 = nn.Linear(2*self.iter_dim,len(num_y_list))
        self.bias = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.weight = nn.Linear(len(seq_y_list)*60+len(num_y_list),1)
        self.avg_pool_1 = AvgPool1d(kernel_size=3,stride=1,padding=1)
    def forward(self, x):
        x_seq = x[:,0:60*len(seq_fea_list)]
        x_num = x[:,60*len(seq_fea_list):x.shape[1]]
        x_seq = x_seq.reshape((-1,len(seq_fea_list),60))
        x_seq = torch.transpose(x_seq, 1, 2)
        x_num_repeat = x_num.reshape((-1,1,len(num_fea_list)))
        x_num_repeat = x_num_repeat.repeat((1,60,1))
        
        x_seq = F.elu(self.Linear_1(torch.concat((x_seq,x_num_repeat),dim=-1)/5))
        
        enc_ftrs = self.encoder(torch.transpose(x_seq,1, 2))
        x_seq_1 = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
        x_seq_1 = torch.transpose(x_seq_1, 1, 2)
        
        # print(x_seq_1.shape)
        
        x_seq_1_mean = torch.mean(x_seq_1,dim=1,keepdim=True)
        x_seq_1_mean = x_seq_1_mean.repeat((1,60,1))

        x_seq_1_avg_pool = self.avg_pool_1(torch.transpose(x_seq_1, 1, 2))
        x_seq_1_avg_pool = torch.transpose(x_seq_1_avg_pool,1, 2)
        
        x_seq_1 = F.elu(self.Linear_2(torch.cat((x_seq_1,x_seq_1_mean,x_seq,x_seq_1_avg_pool),dim=-1)/5))
        
        x_seq_out = self.Linear_3(x_seq_1)
        x_seq_out = torch.transpose(x_seq_out, 1, 2)
        x_seq_out = x_seq_out.reshape((-1,60*len(seq_y_list)))
        
        x_num_out = F.elu(self.Linear_4_0(torch.mean(x_seq_1,dim=1)))
        x_num_out = self.Linear_4(x_num_out)
        
        
        # print(self.weight)
        
        return self.weight.weight*(torch.concat((x_seq_out,x_num_out),dim=-1))/3+self.bias.weight/3
    

input_size = x_valid.shape[1]
output_size = y_valid.shape[1]
hidden_size = input_size + output_size

model_single = FFNN_LSTM_999_AVG_ATT(input_size, output_size)
device_ids = list(range(torch.cuda.device_count()))
model = torch.nn.DataParallel(model_single)
zeroout_index.to(device)


model.to(device)


#model = FFNN_UNET_3(input_size, output_size).to(device)
criterion = nn.MSELoss()
criterion_l1 = nn.L1Loss()
criterion_huber = nn.HuberLoss(delta=0.5)

#model.load_state_dict(torch.load(DATA_PATH+'FFNN_LSTM_888_AVG_ATT_1e-15_12.pt'))


from ema_pytorch import EMA
ema = EMA(
    model,
    beta = 0.99,              # exponential moving average factor
    update_after_step = 50,    # only after this number of .update() calls will it start updating
    update_every = 8,          # how often to actually update, to save on compute (updates every 10th .update() call)
)
# model.load_state_dict(torch.load(DATA_PATH+'model.pt'))


# ema = hkl.load(DATA_PATH+'ema.hkl')



optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0002)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=SCHEDULER_FACTOR, patience=SCHEDULER_PATIENCE)

print("Time after all preparations:", format_time(time.time()-ts), flush=True)


Time after processing data: 0:00:00
Time after all preparations: 0:00:01


In [ ]:

import time
time_gap = 0.00000005
# for g in optimizer.param_groups:
#     g['lr'] = g['lr']/2

best_val_loss = float('inf')
best_model_state = None
patience_count = 0
r2score = R2Score(num_outputs=y_valid.shape[1]).to(device)
for epoch in range(1,400):
    gc.collect()
    torch.cuda.empty_cache()
    
    random.shuffle(file_list)
    random.shuffle(file_list_mix)

    #g['lr'] 
    
    if epoch > 9:
        for g in optimizer.param_groups:
            #g['lr'] = max(g['lr']*0.8,1e-4)
            g['lr']  = 1e-4


    print("")
    model.train()
    ema.train()
    total_loss = 0
    steps = 0
    batch_idx = -1
    
    x_train = []
    y_train = []
    batch_num = 105

    x_train = np.zeros((batch_num*384*6,input_size),dtype=np.float32)
    y_train = np.zeros((batch_num*384*6,output_size),dtype=np.float32)

    x_train_mix = np.zeros((batch_num*384*6,input_size),dtype=np.float32)
    y_train_mix = np.zeros((batch_num*384*6,output_size),dtype=np.float32)

    for file_i in range(len(file_list)): 
        
        y_tmp = hkl.load(file_list[file_i][0:30]+'y'+file_list[file_i][31:len(file_list[file_i])])
        x_tmp = hkl.load(file_list[file_i])

        y_tmp_mix = hkl.load(file_list_mix[file_i][0:32]+'y'+file_list_mix[file_i][33:len(file_list_mix[file_i])])
        x_tmp_mix = hkl.load(file_list_mix[file_i])
        #print(file_i%246*384*6,file_i%246*384*6+384*6)
        x_train[file_i%batch_num*384*6:file_i%batch_num*384*6+384*6,:] = x_tmp
        y_train[file_i%batch_num*384*6:file_i%batch_num*384*6+384*6,:] = y_tmp

        x_train_mix[file_i%batch_num*384*6:file_i%batch_num*384*6+384*6,:] = x_tmp_mix
        y_train_mix[file_i%batch_num*384*6:file_i%batch_num*384*6+384*6,:] = y_tmp_mix

        #y_train[:,zeroout_index] = y_train[:,zeroout_index]*0.0
        
        if (file_i+1)%batch_num == 0:
            gc.collect()


            y_train[:,zeroout_index] = y_train[:,zeroout_index]*0.0
            y_train_mix[:,zeroout_index] = y_train_mix[:,zeroout_index]*0.0
          
            random_index = np.random.permutation(x_train.shape[0])
            i1 = 0
            for i in range(x_train.shape[0]//BATCH_SIZE+1):
                time.sleep(time_gap)
                i2 = np.minimum(i1 + BATCH_SIZE, x_train.shape[0])
                if i1 == i2:  # Break the loop if range does not change
                    break
        
                # Convert the current slice of xt to a PyTorch tensor
                inputs = torch.from_numpy(x_train[random_index[i1:i2], :]).to(device)
                batch_idx = batch_idx + 1
                # if batch_idx%n_rounds == 0:
                #     optimizer.zero_grad()
                outputs = model(inputs)
                outputs_y = torch.from_numpy(y_train[random_index[i1:i2], :]).to(device)

                inputs_mix = torch.from_numpy(x_train_mix[random_index[i1:i2], :]).to(device)
                # if batch_idx%n_rounds == 0:
                #     optimizer.zero_grad()
                outputs_mix = model(inputs_mix)
                outputs_y_mix = torch.from_numpy(y_train_mix[random_index[i1:i2], :]).to(device)

                
                loss = 0.08*criterion(outputs,outputs_y)/n_rounds+criterion_l1(outputs,outputs_y)/n_rounds
                loss = loss + 0.3*criterion_l1(outputs_mix,outputs_y_mix)/n_rounds

                #loss = loss + 0.0001*criterion(outputs_mix,outputs_y_mix)/n_rounds、
                loss = loss + 0.0005*criterion(outputs_mix,outputs_y_mix)/n_rounds

                #loss = torch.mean(loss*loss,axis=0)/error_mean
                #loss = torch.mean(loss)
                loss.backward()
                #torch.nn.utils.clip_grad_norm_(model.parameters(), 1e-3)

                if batch_idx%n_rounds == 0:
                    # loss.backward()
                    
                    
                    
                    optimizer.step()
                    #torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)

                    optimizer.zero_grad()
                    ema.update()
                # print(model.weight.weight[0:10])
        
                total_loss += loss.item()
                steps += 1
        
                if (batch_idx + 1) % PRINT_FREQ == 0:
                    current_lr = optimizer.param_groups[0]["lr"]
                    elapsed_time = format_time(time.time() - ts)
                    print(f'  Epoch: {epoch+1}',\
                          # f'  Batch: {batch_idx + 1}',\
                          f'  Train Loss: {total_loss / steps:.4f}',\
                          f'  LR: {current_lr:.1e}',\
                          f'  Time: {elapsed_time}', flush=True)
                    with open('log0.txt', 'a') as file:
                        file.write(f'  Epoch: {epoch+1}  Train Loss: {total_loss / steps:.4f}  LR: {current_lr:.1e} Time: {elapsed_time}' + '\n')
                    total_loss = 0
                    steps = 0
        
                # No need to track gradients for inference
        
                i1 = i2  # Update i1 to the end of the current batch
        
                if i2 >= x_train.shape[0]:
                    break
            #x_train = np.zeros((batch_num*384*6,input_size)),dtype=np.float32)
            #y_train = np.zeros((batch_num*384*6,output_size)),dtype=np.float32)

    model.eval()
    val_loss = 0
    y_true = torch.tensor([], device=device)
    all_outputs = torch.tensor([], device=device)
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader):
            time.sleep(time_gap)

            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            y_true = torch.cat((y_true, labels), 0)
            all_outputs = torch.cat((all_outputs, outputs), 0)
    r2=0
    r2_broken = []
    r2_broken_names = []
    for i in range(368):
        r2_i = r2score(all_outputs[:, i], y_true[:, i])
        if r2_i > 1e-6:
            r2 += r2_i
        else:
            r2_broken.append(i)
            r2_broken_names.append(FEAT_COLS[i])
    r2 /= 368

    avg_val_loss = val_loss / len(val_loader)
    print(f'\nEpoch: {epoch+1}  Val Loss: {avg_val_loss:.4f}  R2 score: {r2:.4f}')
    print(f'{len(r2_broken)} targets were excluded during evaluation of R2 score.')
    with open('log0.txt', 'a') as file:
            file.write(f'\nEpoch: {epoch+1}  Val Loss: {avg_val_loss:.4f}  R2 score: {r2:.4f}' + '\n')
    # print(r2_broken)
    # print(r2_broken_names, flush=True)
    scheduler.step(round(avg_val_loss*2,4))
    
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_state = model.state_dict()
        patience_count = 0
        print("Validation loss decreased, saving new best model and resetting patience counter.")
    else:
        patience_count += 1
        print(f"No improvement in validation loss for {patience_count} epochs.")
        
    if patience_count >= PATIENCE:
        print("Stopping early due to no improvement in validation loss.")
        break
    
    ema.eval()
    val_loss = 0
    y_true = torch.tensor([], device=device)
    all_outputs = torch.tensor([], device=device)
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader):
            time.sleep(time_gap)

            outputs = ema(inputs)
            val_loss += criterion(outputs, labels).item()
            y_true = torch.cat((y_true, labels), 0)
            all_outputs = torch.cat((all_outputs, outputs), 0)
    valid_results = all_outputs.to('cpu').numpy()
    #hkl.dump(valid_results, DATA_PATH+'valid_full_FFNN_LSTM_6_AVG_2_'+str(epoch)+'.hkl', compression='gzip')
    r2=0
    r2_broken = []
    r2_broken_names = []
    for i in range(368):
        r2_i = r2score(all_outputs[:, i], y_true[:, i])
        if r2_i > 1e-6:
            r2 += r2_i
        else:
            r2_broken.append(i)
            r2_broken_names.append(FEAT_COLS[i])
    r2 /= 368
    
    avg_val_loss = val_loss / len(val_loader)
    print(f'\nEpoch: {epoch+1}  Val Loss: {avg_val_loss:.4f}  R2 score: {r2:.4f}')
    with open('log0.txt', 'a') as file:
        file.write(f'\nEpoch: {epoch+1}  Val Loss: {avg_val_loss:.4f}  R2 score: {r2:.4f}' + '\n')
    print(f'{len(r2_broken)} targets were excluded during evaluation of R2 score.')
    # print(r2_broken)
    # print(r2_broken_names, flush=True)
    predt = np.zeros([x_test.shape[0], output_size], dtype=np.float32)
    # b = np.concatenate((a, a, a),axis=0)
    if epoch%1==0 and epoch>0:
        i1 = 0
        for i in tqdm(range(10000)):
            time.sleep(time_gap)

            i2 = np.minimum(i1 + BATCH_SIZE, x_test.shape[0])
            if i1 == i2:  # Break the loop if range does not change
                break
    
            # Convert the current slice of xt to a PyTorch tensor
            inputs = torch.from_numpy(x_test[i1:i2, :]).float().to(device)
    
            # No need to track gradients for inference
            with torch.no_grad():
                outputs = ema(inputs)  # Get model predictions
                predt[i1:i2, :] = outputs.cpu().numpy()  # Store predictions in predt
    
            i1 = i2  # Update i1 to the end of the current batch
    
            if i2 >= x_test.shape[0]:
                break
        hkl.dump(predt, DATA_PATH+'FFNN_LSTM_999_AVG_ATT_'+str(epoch)+'_mix_1e-15.hkl', compression='gzip')
        torch.save(model.state_dict(), DATA_PATH+'FFNN_LSTM_999_AVG_ATT_1e-15_'+str(epoch)+'.pt')
        torch.save(ema.state_dict(), DATA_PATH+'FFNN_LSTM_999_AVG_ATT_1e-15_ema_'+str(epoch)+'.pt')

        # hkl.dump(ema,DATA_PATH+'ema.hkl')


  Epoch: 2   Train Loss: 0.5134   LR: 1.0e-03   Time: 0:02:45
  Epoch: 2   Train Loss: 0.5003   LR: 1.0e-03   Time: 0:05:16
  Epoch: 2   Train Loss: 0.4746   LR: 1.0e-03   Time: 0:07:44
  Epoch: 2   Train Loss: 0.4365   LR: 1.0e-03   Time: 0:10:15
  Epoch: 2   Train Loss: 0.4026   LR: 1.0e-03   Time: 0:12:42
  Epoch: 2   Train Loss: 0.3955   LR: 1.0e-03   Time: 0:15:12
  Epoch: 2   Train Loss: 0.3608   LR: 1.0e-03   Time: 0:17:40
  Epoch: 2   Train Loss: 0.3438   LR: 1.0e-03   Time: 0:20:11
  Epoch: 2   Train Loss: 0.3138   LR: 1.0e-03   Time: 0:22:58
  Epoch: 2   Train Loss: 0.3123   LR: 1.0e-03   Time: 0:25:42
  Epoch: 2   Train Loss: 0.3019   LR: 1.0e-03   Time: 0:28:28
  Epoch: 2   Train Loss: 0.3076   LR: 1.0e-03   Time: 0:31:16


In [ ]:
len('autodl-tmp/CD Fragment/c_data_frag_')

In [ ]:
torch.save(ema.state_dict(), DATA_PATH+'FFNN_LSTM_749_AVG_ATT_1e-15_ema_'+str(epoch)+'.pt')


In [ ]:
torch.save(model.state_dict(), DATA_PATH+'FFNN_LSTM_7_AVG_ATT_1e-15_'+str(epoch)+'.pt')


In [ ]:
ema.eval()
val_loss = 0
y_true = torch.tensor([], device=device)
all_outputs = torch.tensor([], device=device)
with torch.no_grad():
    for inputs, labels in tqdm(val_loader):
        time.sleep(time_gap)

        outputs = ema(inputs)
        val_loss += criterion(outputs, labels).item()
        y_true = torch.cat((y_true, labels), 0)
        all_outputs = torch.cat((all_outputs, outputs), 0)

In [ ]:
error_mean

In [ ]:
error_num = 2000
error_mean = 0
for file_i in tqdm(range(error_num)):
    y_tmp = hkl.load(file_list[file_i][0:35]+'y'+file_list[file_i][36:len(file_list[file_i])])
    x_tmp = hkl.load(file_list[file_i])
    y_tmp = torch.from_numpy(y_tmp).to(device)
    inputs = torch.from_numpy(x_tmp).to(device)
    error = ema(inputs)-y_tmp
    error_mean = error_mean + torch.mean(error*error,dim=0)

In [ ]:
error_mean = error_mean.to('cpu').numpy()/error_num

In [ ]:
(error_mean+1e-2)*10

In [ ]:
np.mean(error_mean)

In [ ]:
hkl.dump(error_mean,'error_mean.hkl')

In [ ]:
for i in range(368):
    error[i] = np.abs(tmp[:,i] - y_valid[:,i]).mean()

In [ ]:
torch.mean(error*error,dim=0)+0

In [ ]:
np.sort(error)

In [ ]:
error[120:147]

In [ ]:
y[:,120:147]

In [ ]:
y_valid[:,120:147]

In [ ]:
for i in range(368):
    tmp_i = tmp[:,i] - y_valid[:,i]
    tmp_i[tmp_i>0]=1
    tmp_i[tmp_i<0]=-1
    error[i] = abs(tmp_i.mean())

In [ ]:
ema.load_state_dict(torch.load(DATA_PATH+'FFNN_LSTM_749_AVG_ATT_1e-15_ema_30.pt'))

In [ ]:
predt = np.zeros([x_test.shape[0], output_size], dtype=np.float32)
# b = np.concatenate((a, a, a),axis=0)

i1 = 0
for i in tqdm(range(10000)):

    i2 = np.minimum(i1 + BATCH_SIZE, x_test.shape[0])
    if i1 == i2:  # Break the loop if range does not change
        break

    # Convert the current slice of xt to a PyTorch tensor
    inputs = torch.from_numpy(x_test[i1:i2, :]).float().to(device)

    # No need to track gradients for inference
    with torch.no_grad():
        outputs = ema(inputs)  # Get model predictions
        predt[i1:i2, :] = outputs.cpu().numpy()  # Store predictions in predt

    i1 = i2  # Update i1 to the end of the current batch

    if i2 >= x_test.shape[0]:
        break

In [ ]:
zeroout_index

In [ ]:
epoch

In [ ]:
outputs = model(inputs)

In [ ]:
outputs.shape

In [ ]:
file_i

In [ ]:
model.eval()
val_loss = 0
y_true = torch.tensor([], device=device)
all_outputs = torch.tensor([], device=device)
with torch.no_grad():
    for inputs, labels in tqdm(val_loader):
        time.sleep(time_gap)

        outputs = model(inputs)
        val_loss += criterion(outputs, labels).item()
        y_true = torch.cat((y_true, labels), 0)
        all_outputs = torch.cat((all_outputs, outputs), 0)
r2=0
r2_broken = []
r2_broken_names = []
for i in range(368):
    r2_i = r2score(all_outputs[:, i], y_true[:, i])
    if r2_i > 1e-6:
        r2 += r2_i
    else:
        r2_broken.append(i)
        r2_broken_names.append(FEAT_COLS[i])
r2 /= 368

avg_val_loss = val_loss / len(val_loader)
print(f'\nEpoch: {epoch+1}  Val Loss: {avg_val_loss:.4f}  R2 score: {r2:.4f}')
print(f'{len(r2_broken)} targets were excluded during evaluation of R2 score.')
with open('log0.txt', 'a') as file:
        file.write(f'\nEpoch: {epoch+1}  Val Loss: {avg_val_loss:.4f}  R2 score: {r2:.4f}' + '\n')
# print(r2_broken)
# print(r2_broken_names, flush=True)
scheduler.step(round(avg_val_loss*2,4))

if avg_val_loss < best_val_loss:
    best_val_loss = avg_val_loss
    best_model_state = model.state_dict()
    patience_count = 0
    print("Validation loss decreased, saving new best model and resetting patience counter.")
else:
    patience_count += 1
    print(f"No improvement in validation loss for {patience_count} epochs.")
    
if patience_count >= PATIENCE:
    print("Stopping early due to no improvement in validation loss.")
    break

ema.eval()
val_loss = 0
y_true = torch.tensor([], device=device)
all_outputs = torch.tensor([], device=device)
with torch.no_grad():
    for inputs, labels in tqdm(val_loader):
        time.sleep(time_gap)

        outputs = ema(inputs)
        val_loss += criterion(outputs, labels).item()
        y_true = torch.cat((y_true, labels), 0)
        all_outputs = torch.cat((all_outputs, outputs), 0)
valid_results = all_outputs.to('cpu').numpy()
#hkl.dump(valid_results, DATA_PATH+'valid_full_FFNN_LSTM_6_AVG_2_'+str(epoch)+'.hkl', compression='gzip')
r2=0
r2_broken = []
r2_broken_names = []
for i in range(368):
    r2_i = r2score(all_outputs[:, i], y_true[:, i])
    if r2_i > 1e-6:
        r2 += r2_i
    else:
        r2_broken.append(i)
        r2_broken_names.append(FEAT_COLS[i])
r2 /= 368

avg_val_loss = val_loss / len(val_loader)
print(f'\nEpoch: {epoch+1}  Val Loss: {avg_val_loss:.4f}  R2 score: {r2:.4f}')
with open('log0.txt', 'a') as file:
    file.write(f'\nEpoch: {epoch+1}  Val Loss: {avg_val_loss:.4f}  R2 score: {r2:.4f}' + '\n')

In [ ]:
ema.eval()
val_loss = 0
y_true = torch.tensor([], device=device)
all_outputs = torch.tensor([], device=device)
with torch.no_grad():
    for inputs, labels in tqdm(val_loader):
        time.sleep(time_gap)

        outputs = ema(inputs)
        val_loss += criterion(outputs, labels).item()
        y_true = torch.cat((y_true, labels), 0)
        all_outputs = torch.cat((all_outputs, outputs), 0)
valid_results = all_outputs.to('cpu').numpy()
#hkl.dump(valid_results, DATA_PATH+'valid_full_FFNN_LSTM_6_AVG_2_'+str(epoch)+'.hkl', compression='gzip')
r2=0
r2_broken = []
r2_broken_names = []
for i in range(368):
    r2_i = r2score(all_outputs[:, i], y_true[:, i])
    if r2_i > 1e-6:
        r2 += r2_i
    else:
        r2_broken.append(i)
        r2_broken_names.append(FEAT_COLS[i])
r2 /= 368

avg_val_loss = val_loss / len(val_loader)
print(f'\nEpoch: {epoch+1}  Val Loss: {avg_val_loss:.4f}  R2 score: {r2:.4f}')
with open('log0.txt', 'a') as file:
    file.write(f'\nEpoch: {epoch+1}  Val Loss: {avg_val_loss:.4f}  R2 score: {r2:.4f}' + '\n')